In [2]:
!date

Mon Oct  1 10:48:45 CST 2018


In [3]:
import os
import os.path as osp

import requests as req
import pandas as pd

In [4]:
from urllib.parse import urljoin

In [5]:
from pprint import pprint

In [6]:
from io import BytesIO, StringIO

In [7]:
import numpy as np

In [8]:
np.random.seed(999)

In [9]:
def api_path(api_base, *args):
    res = api_base
    for arg in args:
        if arg.startswith('/'):
            res = osp.join(res, arg[1:])
        else:
            res = osp.join(res, arg)
    return res

In [10]:
api_base =  "https://unstats.un.org"

In [11]:
api_path(api_base, "/SDGAPI", "/v1/sdg/Series/List")

'https://unstats.un.org/SDGAPI/v1/sdg/Series/List'

In [12]:
j = req.get(api_path(api_base, "/SDGAPI", "/v1/sdg/Series/List"), params={'allreleases': False}).json()

In [13]:
len(j)

355

In [14]:
_is = np.random.randint(0, len(j), size=10)

In [15]:
for i in _is:
    pprint(j[i])
    print()

{'code': 'EN_WBE_NDWTL',
 'description': 'Nationally derived extent of wetlands (square kilometres)',
 'goal': ['6'],
 'indicator': ['6.6.1'],
 'release': '2018.Q4.G.01',
 'target': ['6.6'],
 'uri': '/v1/sdg/Series/EN_WBE_NDWTL'}

{'code': 'SI_COV_DISAB',
 'description': 'Proportion of population with severe disabilities receiving '
                'disability cash benefit (%)',
 'goal': ['1'],
 'indicator': ['1.3.1'],
 'release': '2018.Q4.G.01',
 'target': ['1.3'],
 'uri': '/v1/sdg/Series/SI_COV_DISAB'}

{'code': 'ER_PTD_MOTN',
 'description': 'Average proportion of Mountain Key Biodiversity Areas (KBAs) '
                'covered by protected areas (%)',
 'goal': ['15'],
 'indicator': ['15.4.1'],
 'release': '2018.Q4.G.01',
 'target': ['15.4'],
 'uri': '/v1/sdg/Series/ER_PTD_MOTN'}

{'code': 'VC_VAW_PHYPYV',
 'description': 'Proportion of children aged 1-14 years who experienced '
                'physical punishment and/or psychological aggression by '
                'caregivers in

In [16]:
j[:10]

[{'goal': ['15', '15'],
  'target': ['15.a', '15.b'],
  'indicator': ['15.a.1', '15.b.1'],
  'release': '2018.Q4.G.01',
  'code': 'DC_ODA_BDVDL',
  'description': 'Total official development assistance for biodiversity, by donor countries (millions of constant 2016 United States dollars)',
  'uri': '/v1/sdg/Series/DC_ODA_BDVDL'},
 {'goal': ['3'],
  'target': ['3.b'],
  'indicator': ['3.b.2'],
  'release': '2018.Q4.G.01',
  'code': 'DC_TOF_HLTHNT',
  'description': 'Total official development assistance to medical research and basic heath sectors, net disbursement, by recipient countries (millions of constant 2016 United States dollars)',
  'uri': '/v1/sdg/Series/DC_TOF_HLTHNT'},
 {'goal': ['9'],
  'target': ['9.a'],
  'indicator': ['9.a.1'],
  'release': '2018.Q4.G.01',
  'code': 'DC_TOF_INFRAL',
  'description': 'Total official flows for infrastructure, by recipient countries (millions of constant 2016 United States dollars)',
  'uri': '/v1/sdg/Series/DC_TOF_INFRAL'},
 {'goal': ['4'],

In [18]:
csv_api = api_path(api_base, "/SDGAPI", "/v1/sdg/Series/DataCSV")

In [23]:
csv = req.post(csv_api, data={'seriesCodes': "DC_ODA_BDVDL"}, stream=True)

In [24]:
with open('../source/DC_ODA_BDVDL.csv', 'wb') as f:
    for chunk in csv.iter_content():
        f.write(chunk)
    f.close()

In [26]:
df = pd.read_csv('../source/DC_ODA_BDVDL.csv')

In [27]:
df.duplicated(subset=['GeoAreaCode', 'TimePeriod']).any()  #hmm..

True

In [28]:
from tqdm import tqdm

In [29]:
for v in tqdm(j):
    code = v['code']
    csv = req.post(csv_api, data={'seriesCodes': code}, stream=True)
    
    with open(f'../source/{code}.csv', 'wb') as f:
        for chunk in csv.iter_content():
            f.write(chunk)
        f.close()

100%|██████████| 355/355 [12:42:48<00:00, 128.93s/it]  


In [39]:
df = pd.read_csv('../source/DC_ODA_BDVDL.csv')

In [40]:
df.head()

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,Source,FootNote,Nature,Units,[Reporting Type]
0,15.0,15.a,15.a.1,DC_ODA_BDVDL,Total official development assistance for biod...,36.0,Australia,2002.0,11.78529,2002.0,"Creditor Reporting System (CRS) database, 2018...","Commitments; Based on OECD, CRS databased, 201...",NaN,CON_USD,G
1,15.0,15.a,15.a.1,DC_ODA_BDVDL,Total official development assistance for biod...,36.0,Australia,2003.0,0.55788,2003.0,"Creditor Reporting System (CRS) database, 2018...","Commitments; Based on OECD, CRS databased, 201...",NaN,CON_USD,G
2,15.0,15.a,15.a.1,DC_ODA_BDVDL,Total official development assistance for biod...,36.0,Australia,2004.0,0.84451,2004.0,"Creditor Reporting System (CRS) database, 2018...","Commitments; Based on OECD, CRS databased, 201...",NaN,CON_USD,G
3,15.0,15.a,15.a.1,DC_ODA_BDVDL,Total official development assistance for biod...,36.0,Australia,2005.0,15.32823,2005.0,"Creditor Reporting System (CRS) database, 2018...","Commitments; Based on OECD, CRS databased, 201...",NaN,CON_USD,G
4,15.0,15.a,15.a.1,DC_ODA_BDVDL,Total official development assistance for biod...,36.0,Australia,2006.0,66.93870,2006.0,"Creditor Reporting System (CRS) database, 2018...","Commitments; Based on OECD, CRS databased, 201...",NaN,CON_USD,G


In [41]:
df['Indicator'].unique()

array(['15.a.1', '15.b.1', nan], dtype=object)

In [42]:
df = df.dropna(how='all')

In [43]:
df['Indicator'].unique()

array(['15.a.1', '15.b.1'], dtype=object)

In [46]:
df['TimePeriod'].unique()

array([2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009., 2010.,
       2011., 2012., 2013., 2014., 2015., 2016.])

In [47]:
df['Time_Detail'].unique()

array([2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009., 2010.,
       2011., 2012., 2013., 2014., 2015., 2016.])

In [49]:
(df['TimePeriod'] == df['Time_Detail']).all()

True

In [45]:
df.duplicated(subset=['GeoAreaCode', 'TimePeriod']).any()

True

In [50]:
# why there are duplicates

In [53]:
df.duplicated(subset=['GeoAreaCode', 'TimePeriod'])

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
670     True
671     True
672     True
673     True
674     True
675     True
676     True
677     True
678     True
679     True
680     True
681     True
682     True
683     True
684     True
685     True
686     True
687     True
688     True
689     True
690     True
691     True
692     True
693     True
694     True
695     True
696     True
697     True
698     True
699     True
Length: 700, dtype: bool

In [54]:
df.loc[670]

Goal                                                                15
Target                                                            15.b
Indicator                                                       15.b.1
SeriesCode                                                DC_ODA_BDVDL
SeriesDescription    Total official development assistance for biod...
GeoAreaCode                                                        756
GeoAreaName                                                Switzerland
TimePeriod                                                        2016
Value                                                          119.813
Time_Detail                                                       2016
Source               Creditor Reporting System (CRS) database, 2018...
FootNote             Commitments; Based on OECD, CRS databased, 201...
Nature                                                             NaN
Units                                                          CON_USD
[Repor

In [55]:
df[(df.GeoAreaName == "Switzerland") & (df.TimePeriod == 2016)]

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,Source,FootNote,Nature,Units,[Reporting Type]
404,15.0,15.a,15.a.1,DC_ODA_BDVDL,Total official development assistance for biod...,756.0,Switzerland,2016.0,119.81321,2016.0,"Creditor Reporting System (CRS) database, 2018...","Commitments; Based on OECD, CRS databased, 201...",NaN,CON_USD,G
670,15.0,15.b,15.b.1,DC_ODA_BDVDL,Total official development assistance for biod...,756.0,Switzerland,2016.0,119.81321,2016.0,"Creditor Reporting System (CRS) database, 2018...","Commitments; Based on OECD, CRS databased, 201...",NaN,CON_USD,G


In [57]:
df.loc[404] == df.loc[670]

Goal                  True
Target               False
Indicator            False
SeriesCode            True
SeriesDescription     True
GeoAreaCode           True
GeoAreaName           True
TimePeriod            True
Value                 True
Time_Detail           True
Source                True
FootNote              True
Nature               False
Units                 True
[Reporting Type]      True
dtype: bool